- Find out what factors most directly impact salaries (title, location, department, etc.). In this case, we do not want to predict mean salary as would be done in a regression. Your boss believes that salary is better represented in categories than continuously
- Test, validate, and describe your models. What factors predict salary category? How do your models perform?
- Prepare a presentation for your Principal detailing your analysis.

In [1]:
# Import relevant libraries
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from spacy import English
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import linear_model
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from IPython.core.display import HTML
from __future__ import unicode_literals

%matplotlib inline

In [2]:
path_to_phantom = '//Applications/phantomjs'

# Identify: Problem Statement / Aim

Our aim is to determine the factors that result in higher salaries for a data scientist.

# Acquire: Import Data Using Requests + BeautifulSoup

- Collect data on data science salary trends from a job listings aggregator for your analysis.
- Select and parse data from at least ~1000 postings for jobs, potentially from multiple location searches.

In [22]:
# create a webdriver PhantomJS object
driver = webdriver.PhantomJS(executable_path=path_to_phantom)
driver.set_window_size(1024,768)

In [23]:
# Indeed.com url formats
# http://www.indeed.com/jobs?q=data+scientist&l=New+York&start=10&pp=
# base - http://www.indeed.com/jobs?q=data+scientist&l=
# location - City, separated by +
# page 2 onwards - &start=10&pp=
# full url - base+location+page

url_top = 'http://www.indeed.com/jobs?q=data+scientist&l='
location = ['New+York', 'Seattle', 'San+Francisco', 'Boston']

In [35]:
# in the search results, each result is wrapped in a div class=' row result'

# for all results we want to get back
# company, span class='company'
# jobtitle, link class='jobtitle'
# location, span class='location'
# summary, span class='summary'
# salary, td class="snip", nobr with a $
# we need to account for missing data

# write a function that can retrieve job title, company, location, summary, and salary for each result
# if there are blanks, return '' for the first 4, and np.nan for salary
def get_details(each_item):
    try:
        salary_text = each_item.find('td', class_='snip').find('nobr').text
        salary_text = salary_text.split()
        salary = float(salary_text[0].strip('$').replace(',',''))
        try:
            job_title = each_item.find('h2', class_='jobtitle').text.strip('\n')
        except:
            job_title = ''

        try:
            company = each_item.find('span', class_='company').text.strip()
        except:
            company = ''

        try:
            location = each_item.find('span', class_='location').text.strip()
        except:
            location = ''

        try:
            summary = each_item.find('span', class_='summary').text.strip()
        except:
            summary = ''
            
        return [job_title, company, location, summary, salary]
    except:
        return 'skip'
    
    
    

In [41]:
entries_required = 100
entries_per_loc = entries_required / len(location)

In [ ]:
data = []
for x in location:
    page = 1
    one_loc = []
    while len(one_loc) <= entries_per_loc:
        full_url = url_top + x
        if page != 1:
            page_url = '&start=' + str((page-1)*10) + '&pp='
            full_url = full_url + page_url
        #print full_url
        page += 1
        driver.get(full_url)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        for item in soup.findAll('div', class_=' row result'):
            if get_details(item) != 'skip':
                one_loc.append(get_details(item))
    #print one_loc
    data.extend(one_loc)
print 'done'

In [38]:
df = pd.DataFrame(data, columns=['job_title', 'company', 'location','summary','salary'])

In [40]:
df

,job_title,company,location,summary,salary
0,"Modeler/Data Scientist, Policy and Strategy",CONSUMER AFFAIRS,"Manhattan, NY","For Assignment Level I (only physical, biologi...",59708.0
1,Top 1% Business Intelligence Analyst,Consulting.com,"New York, NY","We're looking for somebody who's versatile, ca...",150000.0
2,Data Scientist,Big Cloud,"New York, NY","An exciting, innovative company in the heart o...",150000.0
3,Data Scientist,Prime Team Partners,"Seattle, WA 98119 (Queen Anne area)",Prime Team Partners is currently partnering wi...,150000.0
4,"Machine Learning Engineer - Big Data, Python, R",HERO.Jobs,"Seattle, WA","Build big data, machine learning applications ...",160000.0
5,Business Intelligence Scientist,All-In Analytics,"Seattle, WA","SQL, Digital Analytics, eCommerce, Big Data, A...",100000.0
6,Data scientist,Even,"Oakland, CA",Collaborate with our engineers on our internal...,34142.0
7,UC Berkeley - BACEM Computer Support Analyst,Lawrence Berkeley National Laboratory,"Berkeley, CA 94720","QB3 scientists make. Build, debugs and optimiz...",67500.0
8,Data Scientist,Workbridge Associates,"Redwood City, CA","With millions of downloads, this leader in onl...",100000.0
9,Data Scientist (Machine Learning),Corporate Labs Technology,"Brisbane, CA",Machine Learning Developer. Fast growing socia...,110000.0


In [11]:
df.describe(include='all')

,job_title,company,location,summary,salary
count,972,972,972,972,53.000000
unique,625,586,134,897,NaN
top,Data Scientist,Amazon Corporate LLC,"New York, NY",UW Medicine’s mission is to improve the health...,NaN
freq,165,59,156,7,NaN
mean,NaN,NaN,NaN,NaN,102634.339623
std,NaN,NaN,NaN,NaN,55532.974045
min,NaN,NaN,NaN,NaN,45.000000
25%,NaN,NaN,NaN,NaN,67500.000000
50%,NaN,NaN,NaN,NaN,110000.000000
75%,NaN,NaN,NaN,NaN,150000.000000


# Parse: Clean & Organize Data

# Model: Perform Logistic Regression

# Evaluate: Logistic Regression
# Bonus: Countvectorizer, Regularization Parameters

# Present: Write a report for your audience addressing findings & recommendations